In [1]:
import sys
from pathlib import Path
wdir = Path.cwd().parent.absolute()
sys.path.insert(1, str(wdir))

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_ChanRed, CapsEEGNet, TCNet
import torch
import os

In [2]:
model = EEGNet(nb_classes = 2, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = 0
for name, param in model.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: block1.0.weight, Parameters: 1024
Layer: block1.1.weight, Parameters: 64
Layer: block1.1.bias, Parameters: 64
Layer: block1.2.weight, Parameters: 7168
Layer: block1.3.weight, Parameters: 512
Layer: block1.3.bias, Parameters: 512
Layer: block2.0.depthwise.weight, Parameters: 8192
Layer: block2.0.pointwise.weight, Parameters: 32768
Layer: block2.1.weight, Parameters: 64
Layer: block2.1.bias, Parameters: 64
Layer: dense.weight, Parameters: 512
Layer: dense.bias, Parameters: 2
Total Parameters: 50946


In [3]:
model_SEED = EEGNet_ChanRed(nb_classes = 3, Chans = 62, InnerChans=18, Samples = 200, dropoutRate = 0.1, kernLength = 25, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = 0
for name, param in model_SEED.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: chan_reduction.weight, Parameters: 1116
Layer: chan_reduction.bias, Parameters: 18
Layer: block1.0.weight, Parameters: 1600
Layer: block1.1.weight, Parameters: 64
Layer: block1.1.bias, Parameters: 64
Layer: block1.2.weight, Parameters: 9216
Layer: block1.3.weight, Parameters: 512
Layer: block1.3.bias, Parameters: 512
Layer: block2.0.depthwise.weight, Parameters: 8192
Layer: block2.0.pointwise.weight, Parameters: 32768
Layer: block2.1.weight, Parameters: 64
Layer: block2.1.bias, Parameters: 64
Layer: dense.weight, Parameters: 1152
Layer: dense.bias, Parameters: 3
Total Parameters: 55345


In [4]:
x = torch.randn(8, 1, 62, 200)
y = model_SEED(x)
print(y.size())

torch.Size([8, 3])


/Users/juliengadonneix/anaconda3/envs/intern3A/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1712608633180/work/aten/src/ATen/native/Convolution.cpp:1032.)
  return F.conv2d(input, weight, bias, self.stride,


In [5]:
model_capseegnet = CapsEEGNet(nb_classes=2, Chans=14)

total_params = 0
for name, param in model_capseegnet.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: block_1_2.0.weight, Parameters: 512
Layer: block_1_2.1.weight, Parameters: 8
Layer: block_1_2.1.bias, Parameters: 8
Layer: block_1_2.3.weight, Parameters: 224
Layer: block_1_2.4.weight, Parameters: 16
Layer: block_1_2.4.bias, Parameters: 16
Layer: primaryCaps.caps.weight, Parameters: 24576
Layer: primaryCaps.caps.bias, Parameters: 256
Layer: primaryCaps.caps2.weight, Parameters: 69632
Layer: primaryCaps.caps2.bias, Parameters: 256
Layer: emotionCaps.W, Parameters: 1048576
Layer: fc.weight, Parameters: 16
Layer: fc.bias, Parameters: 1
Total Parameters: 1144097


In [6]:
x = torch.randn(8, 1, 14, 128)
y = model_capseegnet(x)
print(y.size())
print(y)

	 torch.Size([8, 16, 1, 128])
	 torch.Size([8, 256, 1, 128])
	 torch.Size([8, 272, 1, 128])
	 torch.Size([8, 256, 1, 128])
	 torch.Size([8, 4096, 8])
torch.Size([8, 2])
tensor([[0.5018, 0.4982],
        [0.4992, 0.5008],
        [0.4999, 0.5001],
        [0.4991, 0.5009],
        [0.4998, 0.5002],
        [0.4985, 0.5015],
        [0.4992, 0.5008],
        [0.4981, 0.5019]], grad_fn=<SoftmaxBackward0>)


In [7]:
model_tcnet = TCNet(nb_classes=2, device='cpu', Chans=14)


total_params = 0
for name, param in model_tcnet.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: PatchPartition.weight, Parameters: 5376
Layer: PatchPartition.bias, Parameters: 32
Layer: primaryCaps.caps.weight, Parameters: 3538944
Layer: primaryCaps.caps.bias, Parameters: 384
Layer: primaryCaps.caps2.weight, Parameters: 163840
Layer: primaryCaps.caps2.bias, Parameters: 256
Layer: emotionCaps.W, Parameters: 65536
Total Parameters: 3774368


/Users/juliengadonneix/anaconda3/envs/intern3A/lib/python3.9/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [8]:
x = torch.randn(8, 14, 48, 128)
out = model_tcnet(x)
print(out.shape)

torch.Size([8, 32, 16, 32])
torch.Size([8, 256, 2, 4])
	 torch.Size([8, 256, 2, 4])
	 torch.Size([8, 384, 2, 4])
	 torch.Size([8, 640, 2, 4])
	 torch.Size([8, 256, 2, 4])
	 torch.Size([8, 256, 8])
torch.Size([8, 256, 8])
torch.Size([8, 2, 16])
torch.Size([8, 2])


In [9]:
print(os.cpu_count())
print(torch.cuda.device_count()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)
print(device.type == 'cuda')
if device.type == 'cuda':
    properties = torch.cuda.get_device_properties(device)
    print(properties)

8
0
mps
False


In [10]:
import subprocess
import plistlib

def get_gpu_info():
    """Get information about the GPU on macOS."""
    # Run system_profiler command and capture the output
    result = subprocess.run(['system_profiler', '-xml', 'SPDisplaysDataType'], capture_output=True)
    if result.returncode == 0:
        # Parse the XML output
        display_info = plistlib.loads(result.stdout)
        # Extract GPU information
        gpu_count = len(display_info[0]['_items'])
        return gpu_count
    else:
        print("Error:", result.stderr)
        return None

gpu_count = get_gpu_info()
if gpu_count is not None:
    print("Number of GPUs:", gpu_count)


Number of GPUs: 1


In [11]:
import time
import torch

start_time = time.time()

# syncrocnize time with cpu, otherwise only time for oflaoding data to gpu would be measured
torch.mps.synchronize()

a = torch.ones(4000,4000, device="mps")
for _ in range(200):
   a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.40522313117980957


In [12]:
t = torch.randn(8,8)
print(t)
torch.reshape(t, (-1, 2, 8))

tensor([[-1.2654, -1.4659,  0.6567, -0.9974,  0.2072,  1.4428,  1.0412,  0.9358],
        [ 1.3798, -0.3627, -0.0027, -0.3838, -1.0778, -0.4006,  0.7975,  0.7431],
        [ 0.3590,  0.2300, -0.3154, -1.1586, -1.2288, -1.5236, -1.7418,  0.4773],
        [-0.1641, -0.7577,  0.1533, -1.1018,  1.2366, -0.8071, -0.2263,  0.6648],
        [ 0.4122,  1.2468, -1.4503, -0.9990,  1.4388,  1.2958, -2.4612,  0.8197],
        [ 0.6530,  0.5259, -0.8077, -1.8335,  1.9630, -0.6820,  0.6334, -0.5389],
        [ 0.5827, -1.5218,  0.2630, -0.0145, -1.8279,  0.2240, -0.6383, -0.5819],
        [ 0.2965, -0.2551,  0.7696, -0.4535, -1.1237,  0.6824, -0.1904,  0.3850]])


tensor([[[-1.2654, -1.4659,  0.6567, -0.9974,  0.2072,  1.4428,  1.0412,
           0.9358],
         [ 1.3798, -0.3627, -0.0027, -0.3838, -1.0778, -0.4006,  0.7975,
           0.7431]],

        [[ 0.3590,  0.2300, -0.3154, -1.1586, -1.2288, -1.5236, -1.7418,
           0.4773],
         [-0.1641, -0.7577,  0.1533, -1.1018,  1.2366, -0.8071, -0.2263,
           0.6648]],

        [[ 0.4122,  1.2468, -1.4503, -0.9990,  1.4388,  1.2958, -2.4612,
           0.8197],
         [ 0.6530,  0.5259, -0.8077, -1.8335,  1.9630, -0.6820,  0.6334,
          -0.5389]],

        [[ 0.5827, -1.5218,  0.2630, -0.0145, -1.8279,  0.2240, -0.6383,
          -0.5819],
         [ 0.2965, -0.2551,  0.7696, -0.4535, -1.1237,  0.6824, -0.1904,
           0.3850]]])